In [1]:
!pip install torchinfo --quiet

In [2]:
import torch  # Импортируем библиотеку PyTorch — фреймворк для построения и обучения нейронных сетей.
import torch.nn as nn  # Импортируем модуль для работы с готовыми слоями нейросети (например, свёртка, полносвязный слой).
import torch.nn.functional as F  # Импортируем функциональный модуль — здесь доступны функции активации, свёртки и т.п.
import torchinfo  # Импортируем torchinfo — библиотеку, которая позволяет красиво выводить информацию о модели (размеры входов, выходов, количество параметров и т.д.)


In [6]:
class SimpleCNN(nn.Module):  # Определяем класс нейросети, наследующий от базового класса nn.Module (все нейросети в PyTorch так строятся).
    def __init__(self):  # Метод инициализации (конструктор). Здесь создаются и настраиваются слои нейросети.
        super(SimpleCNN, self).__init__()  # Вызываем конструктор базового класса (nn.Module), чтобы всё работало корректно.

        # Определяем первый свёрточный слой:
        # nn.Conv2d — слой двумерной свёртки (обычно используется для обработки изображений).
        # Аргументы:
        # 1 — количество входных каналов (у черно-белого изображения 1 канал, у цветного — 3).
        # 32 — количество выходных каналов (то есть фильтров; больше фильтров — больше признаков извлекается).
        # kernel_size=3 — размер фильтра: 3x3 пикселя.
        # padding=1 — добавим по 1 пикселю с каждой стороны изображения, чтобы размер после свёртки не уменьшился.
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)

        # Определяем слой подвыборки (пулинга):
        # MaxPool2d — слой максимального пулинга (берёт максимум из каждого маленького участка изображения).
        # kernel_size=2 — берёт участки размером 2x2 пикселя.
        # stride=2 — шаг смещения окна тоже 2, т.е. изображение уменьшается в 2 раза по ширине и высоте.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Второй свёрточный слой:
        # Входов уже 32 (из предыдущего conv1), выходов — 64 (в 2 раза больше признаков).
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Добавим два полносвязных слоя в класс SimpleCNN:

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        # nn.Linear — это полносвязный (fully connected, FC) слой.
        # Аргументы:
        # 64 * 7 * 7 — число входных признаков (features), приходящих из предыдущего слоя.
        # Почему именно 64 * 7 * 7?
        # - После двух свёрточных слоёв с паддингом=1 и двух MaxPool (kernel_size=2, stride=2),
        #   исходное изображение 224x224 уменьшится в 2 раза → 112x112 после первого пулинга,
        #   затем ещё в 2 раза → 56x56 после второго.
        # - Однако в оригинальном коде был `.view(-1, 64 * 7 * 7)` — это значит, что вход должен быть 28x28, а не 224x224.
        # - То есть реальный размер входного изображения должен быть 28x28, а не 224x224.
        # - Тогда после двух пулингов: 28 → 14 → 7, и выходной тензор будет размером [batch_size, 64, 7, 7].
        # - Его надо "выровнять" (flatten) в [batch_size, 64 * 7 * 7] = [batch_size, 3136].
        # - Этот слой принимает 3136 признаков и выдаёт 128 новых признаков — скрытое представление.

        self.fc2 = nn.Linear(128, 10)
        # Второй полносвязный слой — выходной.
        # Принимает 128 признаков со скрытого слоя и выдаёт 10 выходов.
        # Почему 10?
        # - Обычно это означает, что модель решает задачу классификации на 10 классов (например, цифры от 0 до 9 в MNIST).
        # - Каждое значение — это "оценка" вероятности принадлежности к одному из 10 классов.
        # - В конце используется F.log_softmax, чтобы превратить их в логарифмы вероятностей.


    def forward(self, x):  # Метод прямого прохода (forward) — определяет, как данные проходят через слои.
        # Пропускаем вход x через conv1, затем через ReLU (функция активации), затем через MaxPool:
        x = self.pool(F.relu(self.conv1(x)))

        # Повторяем ту же операцию со вторым слоем:
        x = self.pool(F.relu(self.conv2(x)))

        # После двух свёрток и двух пулингов изображение стало меньше, нужно преобразовать тензор в одномерный вектор:
        # x.view(...) — меняет форму тензора.
        # -1 — PyTorch сам подберёт размер батча, 64 * 7 * 7 — размерность признаков.
        x = x.view(-1, 64 * 7 * 7)

        # Далее — применение полносвязного слоя fc1, но он не определён — это вызовет ошибку.
        x = F.relu(self.fc1(x))  # Активируем скрытый слой через ReLU.

        # Выходной слой — применяем логарифм от softmax (логарифм вероятностей принадлежности к каждому классу):
        x = F.log_softmax(self.fc2(x), dim=1)

        return x  # Возвращаем результат — логарифмы вероятностей.

# Пример использования:
model = SimpleCNN()  # Создаём объект модели.

# Используем torchinfo.summary — выводит таблицу с информацией о слоях, параметрах и размерах.
# model — сама модель.
# (1, 224, 224) — размер входного изображения: 1 канал, 224 пикселя высота, 224 ширина.
# batch_dim=0 — первая размерность (по нулям) означает размер батча.
# col_names — какие колонки выводить: размеры входов/выходов, число параметров, размер ядер, количество операций.
# verbose=0 — отключаем подробный текст.
torchinfo.summary(model, (1, 224, 224), batch_dim=0,
    col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'),
    verbose=0)




Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
SimpleCNN                                [1, 1, 224, 224]          [64, 10]                  --                        --                        --
├─Conv2d: 1-1                            [1, 1, 224, 224]          [1, 32, 224, 224]         320                       [3, 3]                    16,056,320
├─MaxPool2d: 1-2                         [1, 32, 224, 224]         [1, 32, 112, 112]         --                        2                         --
├─Conv2d: 1-3                            [1, 32, 112, 112]         [1, 64, 112, 112]         18,496                    [3, 3]                    232,013,824
├─MaxPool2d: 1-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        2                         --
├─Linear: 1-5                            [1, 3136]                 [64, 128]            

In [7]:
# torchinfo.summary вывёл подробную таблицу по слоям нейросети. Вот объяснение каждой строки и столбца:

# Модель: SimpleCNN — имя нашей модели.

# Вход: [1, 1, 224, 224]
# - Batch size: 1 (одна картинка за раз)
# - Каналов: 1 (чёрно-белое изображение)
# - Высота и ширина: 224x224 пикселя

# ├─Conv2d: 1-1 — первый сверточный слой
#   Input Shape: [1, 1, 224, 224] — один канал, 224x224 пикселя
#   Output Shape: [1, 32, 224, 224] — 32 канала (фильтра), тот же размер (padding=1)
#   Param #: 320 — (1 вход * 3 * 3 + 1 смещение) * 32 фильтра = 320
#   Kernel Shape: [3, 3] — ядро 3x3
#   Mult-Adds: 16,056,320 — операций умножений и сложений: сколько операций нужно сделать для одного прохода по данным

# ├─MaxPool2d: 1-2 — первый пулинг (подвыборка)
#   Output Shape: [1, 32, 112, 112] — размер уменьшен в 2 раза по ширине и высоте

# ├─Conv2d: 1-3 — второй сверточный слой
#   Input Shape: [1, 32, 112, 112]
#   Output Shape: [1, 64, 112, 112] — стало 64 каналов
#   Param #: 18,496 — (32 входных каналов * 3 * 3 + 1 смещение) * 64 фильтра = 18,496 параметров
#   Mult-Adds: 232,013,824 — огромный объём операций на втором свёрточном слое

# ├─MaxPool2d: 1-4 — второй пулинг
#   Output Shape: [1, 64, 56, 56] — снова уменьшаем размер в 2 раза

# ├─Linear: 1-5 — первый полносвязный слой (fc1)
#   Input Shape: [1, 3136] — 64 * 56 * 56 = 200704, но model.view(-1, 64*7*7) рассчитан на [1, 3136]
#   В коде ошибка: текущая выходная форма — 64 * 56 * 56 = 200704, а ожидается 3136 → несовпадение
#   Output Shape: [64, 128] — батч 64 элементов, каждый со 128 признаками
#   Param #: 401,536 — (3136 входов + 1 смещение) * 128 выходов
#   Mult-Adds: 25,698,304 — вычислительные операции на этом слое

# ├─Linear: 1-6 — второй полносвязный слой (fc2)
#   Input Shape: [1, 128]
#   Output Shape: [64, 10] — 10 выходных классов
#   Param #: 1,290 — (128 входов + 1 смещение) * 10 выходов
#   Mult-Adds: 82,560 — сравнительно немного операций

# ИТОГ:
# Total params: 421,642 — общее количество обучаемых параметров (веса и смещения)
# Все параметры — trainable (могут изменяться при обучении)

# Mult-adds (умножения и сложения): 273.85 МЕГАопераций — это показатель вычислительной нагрузки.

# Estimated Total Size (MB): 21.23
# Input: 0.20 MB — размер входного изображения
# Forward/backward pass: 19.34 MB — память, занимаемая при прохождении данных
# Params size: 1.69 MB — вес всех параметров модели
# Total: ~21 MB — сколько потребуется оперативной памяти на один проход
